In [1]:
import pymysql
import baostock as bs
import pandas as pd

In [2]:
def baostock_data(code,col,sd,ed,fq,ajf):
    #### 登陆baostock系统 ####
    lg = bs.login()
    # 显示登陆返回信息
    # lg.error_code：错误代码，当为“0”时表示成功，当为非0时表示失败
    print('login respond error_code:'+lg.error_code)
    # lg.error_msg：错误信息，对错误的详细解释
    print('login respond  error_msg:'+lg.error_msg)
    
    #### 获取历史K线数据 ####
    # frequency="d"取日k线，adjustflag="3"默认不复权
    # 复权状态(1：后复权， 2：前复权，3：不复权），指数自动为3，不复权
    print('正在获取股票数据：'+code)
    rs = bs.query_history_k_data_plus(code,col,
         start_date=sd, end_date=ed, frequency=fq, adjustflag=ajf)
    print('query_history_k_data_plus respond error_code:'+rs.error_code)
    print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)
    
    #### 打印结果集 ####
    data_list = []
    while (rs.error_code == '0') & rs.next():
        # 获取一条记录，将记录合并在一起
        data_list.append(rs.get_row_data())
    result = pd.DataFrame(data_list, columns=rs.fields)
    result['frequency'] = fq
    
    #### 登出系统 ####
    bs.logout()
    return result

In [3]:
def database_create_insert_stock(code,stock,fq,ajf):
    #### 连接database ####
    # conn = pymysql.connect(host=“你的数据库地址”, user=“用户名”,password=“密码”,database=“数据库名”,charset=“utf8”)
    conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='123456' ,charset='utf8')
    # 使用cursor()方法获取操作游标，cursor()得到一个可以执行SQL语句的光标对象
    cursor = conn.cursor()
    #### 选择数据库：stock ####
    Select_DataBase = "use stock;"
    cursor.execute(Select_DataBase)
    table_name = code.replace(".", '')
    
    print('正在创建数据表：'+table_name)
    # 定义要执行的SQL语句，预处理语句创建表
    try:
        sql_create = """create table %s"""% table_name +""" (
                        id int auto_increment,
                        date VARCHAR(20),
                        code VARCHAR(20),
                        open VARCHAR(20),
                        high VARCHAR(20),
                        low VARCHAR(20),
                        close VARCHAR(20),
                        preclose VARCHAR(20),
                        adjustflag VARCHAR(10),
                        volume VARCHAR(20),
                        amount VARCHAR(20),
                        tradestatus VARCHAR(10),
                        turn VARCHAR(20),
                        pctChg VARCHAR(20),
                        peTTM VARCHAR(20),
                        pbMRQ VARCHAR(20),
                        psTTM VARCHAR(20),
                        pcfNcfTTM VARCHAR(20),
                        isST VARCHAR(10),
                        frequency VARCHAR(10),
                        PRIMARY KEY (id),
                        UNIQUE KEY (date,code,adjustflag,frequency)
                        )"""
        # 执行SQL语句
        cursor.execute(sql_create)
    except:
        print(table_name+'数据表已存在！')
        
    sql_insert_d = ''
    sql_insert_w_m = ''
    length = len(stock)
    print('数据量：', length)
    if fq == 'd':
        for i in range(0, length):
            record = tuple(stock.loc[i])
            if sql_insert_d == '':
                #插入数据语句
                sql_insert_d = """INSERT INTO  %s"""% table_name +""" (date,code,open,high,low,close,preclose,
                                  adjustflag,volume,amount,tradestatus,turn,
                                  pctChg,peTTM,pbMRQ,psTTM,pcfNcfTTM,isST,frequency) 
                                  VALUES ('%s','%s','%s','%s','%s','%s','%s',
                                  '%s','%s','%s','%s','%s',
                                  '%s','%s','%s','%s','%s','%s','%s')""" % record
            else:
                sql_insert_d = sql_insert_d + """,('%s','%s','%s','%s','%s','%s','%s',
                                                   '%s','%s','%s','%s','%s',
                                                   '%s','%s','%s','%s','%s','%s','%s')""" % record
        
        try:
            # 执行sql语句
            cursor.execute(sql_insert_d)
            # 提交到数据库执行
            conn.commit()
        except:
            # Rollback in case there is any error
            conn.rollback()
            
        if ajf == '1':
            print('【日线】【后复权[1]】数据成功存入数据库！\n')
        elif ajf == '2':
            print('【日线】【前复权[2]】数据成功存入数据库！\n')
        elif ajf == '3':
            print('【日线】【不复权[3]】数据成功存入数据库！\n')
    else:
        for i in range(0, length):
            record = tuple(stock.loc[i])
            if sql_insert_w_m == '':
                #插入数据语句
                sql_insert_w_m = """INSERT INTO  %s"""% table_name +""" (date,code,open,high,low,close,
                                    volume,amount,adjustflag,turn,pctChg,frequency) 
                                    VALUES ('%s','%s','%s','%s','%s','%s',
                                    '%s','%s','%s','%s','%s','%s')""" % record
            else:
                sql_insert_w_m = sql_insert_w_m + """,('%s','%s','%s','%s','%s','%s',
                                                       '%s','%s','%s','%s','%s','%s')""" % record
        try:
            # 执行sql语句
            cursor.execute(sql_insert_w_m)
            # 提交到数据库执行
            conn.commit()
        except:
            # Rollback in case there is any error
            conn.rollback()
            
        if fq == 'w':
            if ajf == '1':
                print('【周线】【后复权[1]】数据成功存入数据库！\n')
            elif ajf == '2':
                print('【周线】【前复权[2]】数据成功存入数据库！\n')
            elif ajf == '3':
                print('【周线】【不复权[3]】数据成功存入数据库！\n')
        elif fq == 'm':
            if ajf == '1':
                print('【月线】【后复权[1]】数据成功存入数据库！\n')
            elif ajf == '2':
                print('【月线】【前复权[2]】数据成功存入数据库！\n')
            elif ajf == '3':
                print('【月线】【不复权[3]】数据成功存入数据库！\n')
    # 关闭光标对象
    cursor.close()
    # 关闭数据库连接
    conn.close()

In [4]:
def database_replace_updata_stock(code,stock,fq,ajf):
    #### 连接database ####
    # conn = pymysql.connect(host=“你的数据库地址”, user=“用户名”,password=“密码”,database=“数据库名”,charset=“utf8”)
    conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='123456' ,charset='utf8')
    # 使用cursor()方法获取操作游标，cursor()得到一个可以执行SQL语句的光标对象
    cursor = conn.cursor()
    #### 选择数据库：stock ####
    Select_DataBase = "use stock;"
    cursor.execute(Select_DataBase)
    table_name = code.replace(".", '')
        
    sql_insert_d = ''
    sql_insert_w_m = ''
    length = len(stock)
    print('数据量：', length)
    if fq == 'd':
        for i in range(0, length):
            record = tuple(stock.loc[i])
            if sql_insert_d == '':
                #插入数据语句
                sql_insert_d = """REPLACE INTO  %s"""% table_name +""" (date,code,open,high,low,close,preclose,
                                  adjustflag,volume,amount,tradestatus,turn,
                                  pctChg,peTTM,pbMRQ,psTTM,pcfNcfTTM,isST,frequency) 
                                  VALUES ('%s','%s','%s','%s','%s','%s','%s',
                                  '%s','%s','%s','%s','%s',
                                  '%s','%s','%s','%s','%s','%s','%s')""" % record
            else:
                sql_insert_d = sql_insert_d + """,('%s','%s','%s','%s','%s','%s','%s',
                                                   '%s','%s','%s','%s','%s',
                                                   '%s','%s','%s','%s','%s','%s','%s')""" % record
        
        try:
            # 执行sql语句
            cursor.execute(sql_insert_d)
            # 提交到数据库执行
            conn.commit()
        except:
            # Rollback in case there is any error
            conn.rollback()
            
        if ajf == '1':
            print('【日线】【后复权[1]】数据成功存入数据库！\n')
        elif ajf == '2':
            print('【日线】【前复权[2]】数据成功存入数据库！\n')
        elif ajf == '3':
            print('【日线】【不复权[3]】数据成功存入数据库！\n')
    else:
        for i in range(0, length):
            record = tuple(stock.loc[i])
            if sql_insert_w_m == '':
                #插入数据语句
                sql_insert_w_m = """REPLACE INTO  %s"""% table_name +""" (date,code,open,high,low,close,
                                    volume,amount,adjustflag,turn,pctChg,frequency) 
                                    VALUES ('%s','%s','%s','%s','%s','%s',
                                    '%s','%s','%s','%s','%s','%s')""" % record
            else:
                sql_insert_w_m = sql_insert_w_m + """,('%s','%s','%s','%s','%s','%s',
                                                       '%s','%s','%s','%s','%s','%s')""" % record
        try:
            # 执行sql语句
            cursor.execute(sql_insert_w_m)
            # 提交到数据库执行
            conn.commit()
        except:
            # Rollback in case there is any error
            conn.rollback()
            
        if fq == 'w':
            if ajf == '1':
                print('【周线】【后复权[1]】数据成功存入数据库！\n')
            elif ajf == '2':
                print('【周线】【前复权[2]】数据成功存入数据库！\n')
            elif ajf == '3':
                print('【周线】【不复权[3]】数据成功存入数据库！\n')
        elif fq == 'm':
            if ajf == '1':
                print('【月线】【后复权[1]】数据成功存入数据库！\n')
            elif ajf == '2':
                print('【月线】【前复权[2]】数据成功存入数据库！\n')
            elif ajf == '3':
                print('【月线】【不复权[3]】数据成功存入数据库！\n')
    # 关闭光标对象
    cursor.close()
    # 关闭数据库连接
    conn.close()

In [5]:
def database_crt_ist_upd_index(code,stock,table,fq,iu):
    #### 连接database ####
    # conn = pymysql.connect(host=“你的数据库地址”, user=“用户名”,password=“密码”,database=“数据库名”,charset=“utf8”)
    conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='123456' ,charset='utf8')
    # 使用cursor()方法获取操作游标，cursor()得到一个可以执行SQL语句的光标对象
    cursor = conn.cursor()
    #### 选择数据库：stock ####
    Select_DataBase = "use stock;"
    cursor.execute(Select_DataBase)
    
    if table == '0':
        table_name = code.replace(".", '')
    elif table == '1':
        table_name = 'stock_index'
    
    print('正在创建数据表：'+table_name)
    # 定义要执行的SQL语句，预处理语句创建表
    try:
        sql_create = """create table %s"""% table_name +""" (
                        id int auto_increment,
                        date VARCHAR(20) NOT NULL,
                        code VARCHAR(20) NOT NULL,
                        open VARCHAR(20),
                        high VARCHAR(20),
                        low VARCHAR(20),
                        close VARCHAR(20),
                        preclose VARCHAR(20),
                        adjustflag VARCHAR(10) NOT NULL,
                        volume VARCHAR(20),
                        amount VARCHAR(20),
                        pctChg VARCHAR(20),
                        frequency VARCHAR(20),
                        PRIMARY KEY (id),
                        UNIQUE KEY (date,code,adjustflag,frequency)
                        )"""
        # 执行SQL语句
        cursor.execute(sql_create)
    except:
        print(table_name+'数据表已存在！')
    
    length = len(stock)
    if fq == 'd':
        for i in range(0, length):
            record = tuple(stock.loc[i])
            #插入或更新数据语句        
            if iu == '0':
                sql_insert = """INSERT INTO  %s"""% table_name +""" (date, code, open, high, low, close, preclose, adjustflag, volume, amount, pctChg, frequency) 
                                VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')""" % record
            elif iu == '1':
                sql_insert = """REPLACE INTO  %s"""% table_name +""" (date, code, open, high, low, close, preclose, adjustflag, volume, amount, pctChg, frequency) 
                                VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')""" % record
            # 没有设置批量values导入，则try在for循环内部
            try:
                # 执行sql语句
                cursor.execute(sql_insert)
                # 提交到数据库执行
                conn.commit()
            except:
                # Rollback in case there is any error
                conn.rollback()
        print('【日线】【不复权[3]】数据成功存入数据库！\n')
    else:
        for i in range(0, length):
            record = tuple(stock.loc[i])
            #插入或更新数据语句        
            if iu == '0':
                sql_insert = """INSERT INTO  %s"""% table_name +""" (date, code, open, high, low, close, adjustflag, volume, amount, pctChg, frequency) 
                                VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')""" % record
            elif iu == '1':
                sql_insert = """REPLACE INTO  %s"""% table_name +""" (date, code, open, high, low, close, adjustflag, volume, amount, pctChg, frequency) 
                                VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')""" % record
            # 没有设置批量values导入，则try在for循环内部
            try:
                # 执行sql语句
                cursor.execute(sql_insert)
                # 提交到数据库执行
                conn.commit()
            except:
                # Rollback in case there is any error
                conn.rollback()
        if fq == 'w':
            print('【周线】【不复权[3]】数据成功存入数据库！\n')
        elif fq == 'm':
            print('【月线】【不复权[3]】数据成功存入数据库！\n')
            
    # 关闭光标对象
    cursor.close()
    # 关闭数据库连接
    conn.close()